In [1]:
import sys
import glob
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import datetime as dt
import matplotlib.cm as cm
import matplotlib.pyplot as plt

from math import sqrt
from prophet import Prophet
from pmdarima import auto_arima
from xgboost import XGBRegressor
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

from src.utils import util as utl
from itertools import combinations, permutations, product
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

global companies_set 

warnings.filterwarnings('ignore')

# 1 - Estudo de auto regressão dos mercados

## 1.1 - Preparação dos datasets com os valores totais de cada mercado

In [ ]:
door_dir = glob.glob(r"D:\Documentos_D\UFCG\2021.2e\TCC\Project\analise_de_indicador_tcc_ufcg\data\processed\**\*" + ".csv", recursive=True)

df = pd.DataFrame()

for file_path in door_dir:
    temp_df = pd.read_csv(file_path)
    df = df.append(temp_df)

df = df.reset_index().drop(['index'], axis=1)
df['Volume_Venda'] = df['Volume_Venda'].round(2)

In [ ]:
dfm = pd.DataFrame()
temp_df = df[((df.Mercado == 'AN') & (df.Empresa == 'E2'))].reset_index().drop(['index'],axis=1)

dfm['Data'] = temp_df.Data
dfm['Data'] = pd.to_datetime(dfm.Data)
# ----------------------------------------------------AN:
r1_bc = df[((df.Mercado == 'AN') & (df.Empresa == 'E1'))].Volume_Venda.astype(float).to_frame()
r1_br = df[((df.Mercado == 'AN') & (df.Empresa == 'E2'))].Volume_Venda.reset_index().drop(['index'], axis=1).astype(float)

dfm['AN'] = r1_bc.add(r1_br,axis=0)

# ----------------------------------------------------AS:
r2_bi = df[((df.Mercado == 'AS') & (df.Empresa == 'E3'))].Volume_Venda.reset_index().drop(['index'], axis=1).astype(float)
r2_br = df[((df.Mercado == 'AS') & (df.Empresa == 'E2'))].Volume_Venda.reset_index().drop(['index'], axis=1).astype(float)

r2_bt_temp = pd.DataFrame(columns=['Volume_Venda'])
values = []
for i in range(18):
    values.append(0)
r2_bt_temp['Volume_Venda'] = values
r2_bt = df[((df.Mercado == 'AS') & (df.Empresa == 'E4'))].Volume_Venda.reset_index().drop(['index'], axis=1).astype(float)
r2_bt = r2_bt.append(r2_bt_temp).reset_index().drop(['index'], axis=1)

dfm['AS'] = r2_bi.add(r2_br,axis=0)
dfm['AS'] = dfm['AS'] + r2_bt['Volume_Venda']

# ----------------------------------------------------EU:
r3_bi = df[((df.Mercado == 'EU') & (df.Empresa == 'E3'))].Volume_Venda.reset_index().drop(['index'], axis=1).astype(float)
r3_br = df[((df.Mercado == 'EU') & (df.Empresa == 'E2'))].Volume_Venda.reset_index().drop(['index'], axis=1).astype(float)

dfm['EU'] = r3_bi.add(r3_br,axis=0)

# ----------------------------------------------------AL:
r4_br_temp = pd.DataFrame(columns=['Volume_Venda'])
values = []
for i in range(13):
    values.append(0)
r4_br_temp['Volume_Venda'] = values
r4_br = df[((df.Mercado == 'AL') & (df.Empresa == 'E2'))].Volume_Venda.reset_index().drop(['index'], axis=1).astype(float)
r4_br = r4_br.append(r4_br_temp).reset_index().drop(['index'], axis=1)

r4_bt_temp = pd.DataFrame(columns=['Volume_Venda'])
values = []
for i in range(18):
    values.append(0)
r4_bt_temp['Volume_Venda'] = values
r4_bt = df[((df.Mercado == 'AL') & (df.Empresa == 'E4'))].Volume_Venda.reset_index().drop(['index'], axis=1).astype(float)
r4_bt = r4_bt.append(r4_bt_temp).reset_index().drop(['index'], axis=1)

dfm['AL'] = r4_br.add(r4_bt,axis=0)

# ----------------------------------------------------JP:
r5_bi_temp = pd.DataFrame(columns=['Volume_Venda'])
values = []
for i in range(56):
    values.append(0)
r5_bi_temp ['Volume_Venda'] = values
r5_bi = df[((df.Mercado == 'JP') & (df.Empresa == 'E3'))].Volume_Venda.reset_index().drop(['index'], axis=1).astype(float)
r5_bi = r5_bi_temp.append(r5_bi).reset_index().drop(['index'], axis=1)

r5_br = df[((df.Mercado == 'JP') & (df.Empresa == 'E2'))].Volume_Venda.reset_index().drop(['index'], axis=1).astype(float)

r5_bt_temp = pd.DataFrame(columns=['Volume_Venda'])
values = []
for i in range(18):
    values.append(0)
r5_bt_temp ['Volume_Venda'] = values
r5_bt = df[((df.Mercado == 'JP') & (df.Empresa == 'E4'))].Volume_Venda.reset_index().drop(['index'], axis=1).astype(float)
r5_bt = r5_bt.append(r5_bt_temp ).reset_index().drop(['index'], axis=1)

dfm['JP'] = r5_bi.add(r5_br, axis=0)
dfm['JP'] = dfm['JP'] + r5_bt['Volume_Venda']

# ----------------------------------------------------BRL:
r6_br = df[((df.Mercado == 'BRL') & (df.Empresa == 'E2'))].Volume_Venda.reset_index().drop(['index'], axis=1).astype(float)
dfm['BRL'] = r6_br

# ----------------------------------------------------Indefinido:
indefinido = df[((df.Mercado == 'Indefinido'))].Volume_Venda.reset_index().drop(['index'], axis=1).astype(float)
dfm['Indefinido'] = indefinido

dfm = dfm.set_index('Data')

display(dfm.head(5))

In [ ]:
markets_names = ['AN','AS','EU','AL','JP','BRL'] # O mercado 'Indefinido' será desconsiderado, a partir deste momento;
markets_values = []
for market_name in markets_names:  
    values_market_df = dfm[market_name]
    markets_values.append(values_market_df.to_frame())

In [ ]:
# Criando  volume de dados para treinamento do mercado da América Latina ('AL'), à parte:
date_indexes = []
sales_values = []

for market_values_df, market_name in zip(markets_values, markets_names):
    if market_name in ['AL']:
        temp_df =  market_values_df
        for i, row in temp_df.iterrows():
            date = i
            value = row['AL']

            if date < pd.Timestamp('2016-01-01'): # Coletando valores até 01/12/2015;
                date_indexes.append(i)
                sales_values.append(value)

df_AL = pd.DataFrame()
df_AL['Data'] =  date_indexes
df_AL['AL'] =  sales_values
df_AL = df_AL.set_index('Data')
markets_values[3] = df_AL

## 1.2 - Teste de Estacionariedade

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from pandas.plotting import lag_plot, autocorrelation_plot
from statsmodels.tsa.stattools import adfuller

p_values = []

# Verificando estacionariedade com adfuller:
for market_values_df, market_name in zip(markets_values, markets_names):
    series = market_values_df
    X = series.values
    train, test = X[1:len(X)-12], X[len(X)-12:]
    result = adfuller(train, autolag='AIC')
    p_values.append((market_name,np.round(result[0],4),np.round(result[1],4)))

print(p_values)

## 1.3 - Uso da biblioteca AutoARIMA para verificação dos melhores parâmetros de ARIMA para os mercados

In [ ]:
for market_values_df, market_name in zip(markets_values, markets_names):
    print("Mercado: ", market_name)
    X = market_values_df.values
    train, test = X[1:len(X)-12], X[len(X)-12:]
    model = auto_arima(train, start_P=0, start_q=0, stepwise=True, trace=True)

## 1.4 - Execução do modelo de auto regressão Autoreg

In [ ]:
from statsmodels.tsa.ar_model import AutoReg

# Criação de diversas combinações dos parâmetros 'window' e 'lags':
windows_lags = []
for wg in product([1,2,3,4,5,6,7,8,9,10,11,12], repeat = 2):
    windows_lags.append(wg)

maes_autoreg = []
r2s_autoreg = []

results_autoreg ={}

for market_values_df, market_name in zip(markets_values, markets_names): 
    # Separando o dataset entre treino e teste:
    series = market_values_df
    X = series.values
    train, test = X[1:len(X)-12], X[len(X)-12:]

    wgs_autoreg = []
    for wg in windows_lags:
        # Treinando a autoregressão:
        window = wg[0]
        lags = wg[1]

        if lags >= window:
            model = AutoReg(train, lags=lags)
            model_fit = model.fit()
            coef = model_fit.params

            # Caminhando sobre o dataset de teste para comparar o valor real com o predito:
            history = train[len(train)-window:]
            history = [history[i] for i in range(len(history))]
            predictions = []

            try: 
                for t in range(len(test)):
                    length = len(history)
                    lag = [history[i] for i in range(length-window,length)]
                    yhat = coef[0]

                    for d in range(window):
                        yhat += coef[d+1] * lag[window-d-1]

                    obs = test[t]
                    predictions.append(yhat)
                    history.append(obs)
            except:
                print('Window: ', window)
                print('Lags: ', lags )

            mae = mean_absolute_error(test, predictions)
            maes_autoreg.append(mae)

            r2_scr = r2_score(test, predictions)
            r2s_autoreg.append(r2_scr)

            wgs_autoreg.append((wg, r2_scr, mae))
    results_autoreg[market_name] = wgs_autoreg 

# Coletando os melhores resultados do Autoreg para cada mercado, com o respectivo parâmetro utilizado:
res_autoreg = []
for key in results_autoreg.keys():
    temp_order = []
    temp_r2 = []
    temp_mae = []
    for tpl in results_autoreg[key]:
        order = tpl[0]
        r2 = tpl[1]
        mae = tpl[2]

        temp_order.append(order)
        temp_r2.append(r2)
        temp_mae.append(mae)

    max_pos = temp_r2.index(max(temp_r2))
    res_autoreg.append((key, temp_order[max_pos], temp_r2[max_pos], temp_mae[max_pos]))

print(res_autoreg)

## 1.5 - Execução do modelo de auto regressão ARIMA

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

# Criação de lista com combinações, para teste de ordens (p,d,q) no ARIMA. Obteve-se apenas as triplas com o valor d = 0:
orders = []
for ord in product([0,1,2,3,4,5,6,7,8,9,10,11,12], repeat = 3):
    if ord[1] == 0:
        orders.append(ord)
        
i = 0

results_arima = {}
for market_values_df, market_name in zip(markets_values, markets_names):
    if market_name != 'AL':
        print("mercado: ", market_name)
        X = market_values_df[market_name]
        train, test = X[1:len(X)-12], X[len(X)-12:]

        ords_arima = []
        for ord in orders:
            model = ARIMA(train, order=ord)
            
            try:
                res = model.fit()
                predictions = res.predict(start=pd.to_datetime('2021-01-01'), end=pd.to_datetime('2021-12-01'), dynamic=False)
                print("ord: ",ord)
                print("Predis: ", predictions)
            except:
                pass

            mae = mean_absolute_error(test, predictions)
            # maes_arima.append(mae)

            r2_scr = r2_score(test, predictions)
            # r2s_arima.append(r2_scr)

            ords_arima.append((ord, r2_scr, mae))
        
        print(market_name+" finalizado!")

        results_arima[market_name] = ords_arima 

# Coletando os melhores resultados do ARIMA para cada mercado, com o respectivo parâmetro utilizado:   
res_arima = []
for key in results_arima.keys():
    temp_order = []
    temp_r2 = []
    temp_mae = []
    for tpl in results_arima[key]:
        order = tpl[0]
        r2 = tpl[1]
        mae = tpl[2]

        temp_order.append(order)
        temp_r2.append(r2)
        temp_mae.append(mae)

    max_pos = temp_r2.index(max(temp_r2))
    res_arima.append((key, temp_order[max_pos], temp_r2[max_pos], temp_mae[max_pos]))

print(res_arima)

## 1.6 - Execução do modelo de regressão XGBoostRegressor

In [ ]:
def train_test_split(data,perc):
    data = data.values
    n = int(len(data) * (1 - perc))
    return data[:n], data[n:]

def xgb_predict(train, val, model):
    train = np.array(train)
    X, y = train[:, :-1], train[:, -1]

    model.fit(X,y)

    val = np.array(val).reshape(1, -1)
    pred = model.predict(val)
    return pred[0]

def validate(data, perc, model):
    predictions = []
    train, test = train_test_split(data, perc)

    history = [x for x in train]

    for i in range(len(test)):
        test_X, test_y = test[i, :-1], test[i, -1]

        pred = xgb_predict(history, test_X[0], model)
        predictions.append(pred)

        history.append(test[i]) # Histórico, para ir atualizando o treino com os valores mais novos;
    
    error_rmse = mean_squared_error(test[:, -1], predictions, squared=False)
    error_mae = mean_absolute_error(test[:, -1], predictions)
    return error_rmse, error_mae, test[:, -1], predictions

In [ ]:
model = XGBRegressor(objective='reg:squarederror', n_estimators=1000)

for market_values_df, market_name in zip(markets_values, markets_names):

    market_values_df_copy =  market_values_df.copy(deep=True)
    market_values_df_copy['target'] = market_values_df_copy[market_name].shift(-1)
    market_values_df_copy.dropna(inplace=True)

    if market_name != 'AL':
        perc = 0.1
    else:
        perc = 0.25

    rmse, mae,  y, pred = validate(market_values_df_copy, perc, model)

    r2_scr = r2_score(y, pred)

    print("Mercado: ", market_name)
    print("r² score: ", r2_scr)
    print("MAE: ", mae)
    print("RMSE: ", rmse)

## 1.7 - Execução as predições para os anos sem dados históricos (2016 e 2022) com o prophet

In [ ]:
for market_values_df, market_name in zip(markets_values, markets_names):
    market_values_df_copy = market_values_df

    market_values_df_copy['Data'] = market_values_df_copy.index
    cols = market_values_df_copy.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    market_values_df_copy = market_values_df_copy[cols].reset_index(drop=True)
    market_values_df_copy.rename(columns = {'Data':'ds',market_name:'y'}, inplace = True)

    model = Prophet()
    model.fit(market_values_df_copy)

    future = model.make_future_dataframe(periods=13, freq ='M')
    if market_name != 'AL':
        future.drop(future.index[future['ds'] == '2021-12-31'], inplace=True)
    else:
        future.drop(future.index[future['ds'] == '2015-12-31'], inplace=True)

    forecast = model.predict(future)

    fig = model.plot(forecast, xlabel='Período',ylabel='Volume de Venda (t)')
    ax = fig.gca()
    print(market_name)
